# Modelling demand response: comparison of modelling approaches

This notebook contains a comparison of Demand Response modelling approaches. In the following, the terms Demand Response (DR) and Demand Side Management (DSM) are used interchangeably. The focus is on load shifting, but load shedding is included in the model formulations compared as well.

## Purpose and background

* This notebook is based on an implementation of Julian Endres created within the wind note project, see: https://github.com/windnode/SinkDSM_example/blob/master/DSM-Modelling.ipynb
* It's main purpose is to compare different implementations for demand response modelling.
* The modelling approaches considered are the following ones:
    * Zerrahn & Schill (2015)
    * Steurer (2017)
    * Gils (2015)
    * Ladwig (2018)

## Method

To compare the modelling approaches, the following methodology is applied:
1. A simple toy model is set up.
    * There is one backup generation unit and one serving average demand at no marginal costs.
    * DSM units are defined to be a low variable costs option.
    * The parameterization for DSM units is harmonized as far as this is possible for the different modelling approaches in order to isolate effects which are based on different DSM implementations.
2. Different cases are depicted using the toy model in order to compare different DR modelling approaches:
    * Case 1: Simple demand variation
    * Case 2: Demand variations
        * 2a: variations with the same amplitude each
        * 2b: variations with a changing amplitude
        * 2c: variations with changed starting timesteps for load shifts
        * 2d: variations with a longer duration (shift for several subsequent hours)
        * 2e: variations demanding for longer shift times
    * Case 3: Increase in demand (use case for load shedding)
    * Case 4: (asymmetric) variations in generation and constant demand
    * Case 4: variations in demand and generation 
3. Different parameter variations and optional constraints are analyzed, comprising the following:
    * variations in delay time, i.e. shifting time
    * variations in shift time, i.e. interference time
    * variations in DSM costs
    * variations in DSM efficiency
    * variations in recovery time resp. maximum activations
        * basic comparison
        * changed demand pattern for comparison
        * separate comparison of short term limits imposed
    * variations in the amoung of daily activations
    * analysis of additional constraint imposing an overall DSM capacity limit
4. Benchmarks concerning the following elements are carried out
    * objective value
    * runtime

In addition to the very stylized toy model, a more realistic example model is considered as well and other stylized examples are added.

# Package Imports and plot settings
Imports:
* Standard imports
* Import the different implementations for demand response components
* Import module `plotting.py` for extracting results and visualization

Plot settings:<br>
* Register matplotlib converters.
* Adjust matplotlib standard settings for graphs.
* Create a directory to store graphs (if it doesn't already exist).

In [ ]:
import pandas as pd
import numpy as np
import os
import pprint
from math import sqrt

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters

import oemof.solph as solph
from oemof.network.network import Node

# Import the different alternative implementations
import oemof_SinkDSM as DSM_DIW
import oemof_DR_component_DLR_naming_adjusted as DSM_DLR
import oemof_DR_component_DLR_naming_adjusted_shifting_classes as DSM_DLR_ShiftClass
import oemof_DR_component_DLR_naming_adjusted_no_shed as DSM_DLR_NoShed
import oemof_DR_component_IER_naming_adjusted as DSM_IER
import oemof_DR_component_TUD_naming_adjusted as DSM_TUD

# Import module for plotting (results handling)
import plotting as plt_dsm

In [ ]:
# Determine matplotlib settings
register_matplotlib_converters()

SMALL_SIZE = 13
MEDIUM_SIZE = 14
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
## base dataset
plt_dsm.make_directory('graphics')

# Parameter settings
* General parameter settings for controlling the notebooks workflow
* Special parameter settings for DSM parameterization

## General parameter settings
Control **workflow** of notebook:
* *performance_benchmark*: If True, model performance will be evaluated for the different approaches.<br> &rarr; _Note: Since this is quite computation intense, it should be deactivated if it is not the special interest._
* *target_benchmark*: If True, the target function values for the different modelling approaches will be compared.
* *save_figs*: If True, all figures will be saved in the graphics folder.
* *save_results*: If True, overall amounts of demand response activations will be saved to .csv files

In [ ]:
# Parameters to control overall workflow
performance_benchmark = False
target_benchmark = False
save_figs = True
save_results = True

## DSM parameter settings
Define major **parameters concerning demand response modelling**
* *aproaches*: List of the approaches used for demand response modelling
* *addition*: Boolean parameter indicating, whether or not to include an additional "logic" constraint into the other DSM modelling approaches that is similar to equation 10 of Zerrahn & Schill (2015)
* *efficiency*: Consider a pontential efficiency loss for those modelling approaches which depict DSM efficiency (all except for Ladwig 2018).
* *recovery_time*: Consider a pontential recovery time for those modelling approaches which depict it (only Zerrahn & Schill 2015).
* *ActivateYearLimit*: Boolean variable indicating whether or not to use a limit for DSM activations per year (only applicable for Gils 2015).
* *ActivateDayLimit*: Boolean variable indicating whether or not to use a limit for DSM activations per day resp. per rolling window (only applicable for Gils 2015).

Determine **costs for demand response**:
* *include_costs*: If True, (small) variable costs will be included.
* *cost_dsm*: Overall variable costs for demand response which have to be splitted up to up and downwards shifts
* *cost_dsm_up*: Costs for upwards shifts (_defaults to have of the overall costs_)
* *cost_dsm_down*: Costs for downwards shifts (_defaults to have of the overall costs_)

Introduce special control variables for controlling the **workflow**, especially for approach from DLR (Gils 2015):
* *introduce_second_dsm_unit*: If True, a second demand response unit with the same parameterization will be introduced.
* *few_timesteps*: If True, for the simple example the timeset will be limited to 9 timesteps in order to increase readability of the .lp-files
* *use_shifting_classes*: If True, shifting classes will be applied in the approach from DLR (Gils 2015)
* *use_no_shed*: If True, the approach from DLR (Gils 2015) withouth a load shedding implementation will be utilized. 

In [ ]:
# Parameters focussing on demand response modelling
approaches = ["DIW", "DLR", "IER", "TUD"]
addition = False
efficiency = 1
recovery_time = None
ActivateYearLimit = False
ActivateDayLimit = False

# Determine cost consideration
include_costs = True

if include_costs:
    cost_dsm = 0.1

else:
    cost_dsm = 0

# Cost is split half on upwards and downwards shift; shedding gets high costs
cost_dsm_up = cost_dsm/2
cost_dsm_down_shift = cost_dsm/2
cost_dsm_down_shed = 1000 * cost_dsm

In [ ]:
# Control variables for demand response modelling
introduce_second_dsm_unit = False
few_timesteps = True

# Define whether or not to use the shifting classes approach for DLR (Gils 2015) -> not working yet!
use_no_shed = False
use_shifting_classes = True

# Tools for setup, results extraction and visualization of a (toy) energy system model
For the testing, a **toy energy system** is set up including:
- Coal PP
- Wind PP
- PV PP
- DSM Sink
- shortage
- excess

**Rules for DSM parametrization**:

The following rules apply for parameters which are not part of every modelling approach:<br>
* shift (resp. interference) times: These will be defined half of the delay time and symmetrical in the first place.
* additional parameters: These will be defined to be not bounding in the first place.
* optional parameters & constraints: These will be ignored in the first place.
* additional constraints: These will also be ignored in the first place.

## Create and solve the model
A function is defined here for setting up and solving the toy model.

In [ ]:
def create_model(data, datetimeindex, directory, project, approach,
                 delay_time, shed_time, cost_dsm_up, cost_dsm_down_shift, 
                 cost_dsm_down_shed, efficiency,
                 shed_eligibility, shift_eligibility, introduce_second_dsm_unit,
                 **kwargs):
    """ Function to create and solve the model. """
    
    # Control generation units
    include_coal = kwargs.get('include_coal', True)
    include_gas = kwargs.get('include_gas', False)
    nom_cap_coal = kwargs.get('nom_cap_coal', 10000)
    nom_cap_gas = kwargs.get('nom_cap_gas', 10000)
    nom_cap_pv = kwargs.get('nom_cap_pv', 1)
    
    # Special kwargs for DSM modelling (one approach only)
    method = kwargs.get('method', 'delay')
    shift_interval = kwargs.get('shift_interval', 24)
    recovery_time_shift = kwargs.get('recovery_time_shift', None)
    recovery_time_shed = kwargs.get('recovery_time_shed', 24)
    use_no_shed = kwargs.get('use_no_shed', False)
    use_shifting_classes = kwargs.get('use_shifting_classes', False)
    shift_time = kwargs.get('shift_time', delay_time/2)
    start_times = kwargs.get('start_times', [1])
    
    addition = kwargs.get('addition', False)
    fixes = kwargs.get('fixes', False)
    
    ActivateYearLimit = kwargs.get('ActivateYearLimit', False)
    ActivateDayLimit = kwargs.get('ActivateDayLimit', False)

    # ----------------- Energy System ----------------------------
    
    # Create Energy System
    es = solph.EnergySystem(timeindex=datetimeindex)
                           #groupings=[type])
    len_data = len(data.loc[datetimeindex,:])    
    
    Node.registry = es

    # Create Buses
    if include_coal:
        b_coal_1 = solph.Bus(label='bus_coal_1')
    if include_gas:
        b_gas_1 = solph.Bus(label='bus_gas_1')
    b_elec = solph.Bus(label='bus_elec')

    # Create Sources
    if include_coal:
        s_coal_p1 = solph.Source(label='source_coal_p1',
                                 outputs={
                                    b_coal_1: solph.Flow(
                                        nominal_value=nom_cap_coal,
                                        variable_costs=13)}
                                 )
    if include_gas:
        s_gas_p1 = solph.Source(label='source_gas_p1',
                                 outputs={
                                    b_gas_1: solph.Flow(
                                        nominal_value=nom_cap_gas,
                                        variable_costs=25)}
                                 )        

    s_wind = solph.Source(label='wind',
                          outputs={
                              b_elec: solph.Flow(
                                  fix=data['wind'][datetimeindex],
                                  nominal_value=1)}
                          )

    s_pv = solph.Source(label='pv',
                        outputs={
                            b_elec: solph.Flow(
                                fix=data['pv'][datetimeindex],
                                nominal_value=nom_cap_pv)}
                        )

    # Create Transformers
    if include_coal:
        cfp_1 = solph.Transformer(label='pp_coal_1',
                                  inputs={b_coal_1: solph.Flow()},
                                  outputs={
                                      b_elec: solph.Flow(
                                          variable_costs=0)},
                                  conversion_factors={b_elec: 0.4}
                                  )
    if include_gas:
        gfp_1 = solph.Transformer(label='pp_gas_1',
                                  inputs={b_gas_1: solph.Flow()},
                                  outputs={
                                      b_elec: solph.Flow(
                                          variable_costs=0)},
                                  conversion_factors={b_elec: 0.4}
                                  )
    
    # Create DSM units

    # Define kwargs that are identical for all DSM units
    kwargs_all = {'label': 'demand_dsm',
                  'inputs': {b_elec: solph.Flow(variable_costs=0)},
                  'demand': data['demand_el'][datetimeindex],
                  'capacity_up': data['Cap_up'][datetimeindex],
                  'capacity_down': data['Cap_do'][datetimeindex],
                  'delay_time': delay_time,
                  'shed_time': shed_time,
                  'recovery_time_shift': recovery_time_shift,
                  'recovery_time_shed': recovery_time_shed,
                  'cost_dsm_up': cost_dsm_up,
                  'cost_dsm_down_shift': cost_dsm_down_shift,
                  'cost_dsm_down_shed': cost_dsm_down_shed,
                  'efficiency': efficiency,
                  'shed_eligibility': shed_eligibility,
                  'shift_eligibility': shift_eligibility}
    
    # Define kwargs that differ dependent on approach chosen
    
    # Determine recovery / max activations / cumulative shift / shed time dependent on each other
    if recovery_time_shift is not None:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len_data // (delay_time + recovery_time_shift))
        daily_frequency_shift = kwargs.get('daily_frequency_shift', 
                                           24 // (delay_time + recovery_time_shift))
    else:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len_data // delay_time)
        daily_frequency_shift = kwargs.get('daily_frequency_shift', 
                                           24 // delay_time)
    
    if recovery_time_shed is not None:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len_data // (shed_time + recovery_time_shed))
    else:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len_data // shed_time)
    
    cumulative_shift_time = n_yearLimit_shift * shift_time
    cumulative_shed_time = n_yearLimit_shed * shed_time
    
    if not daily_frequency_shift == 0:
        t_dayLimit = kwargs.get('t_dayLimit', 
                                24 / daily_frequency_shift - 1)
    else:
        t_dayLimit = kwargs.get('t_dayLimit', 0) 

    kwargs_dict = {
        'DIW': {'method': method,
                'shift_interval': shift_interval},
                   
        'IER': {'shift_time_up': shift_time,
                'shift_time_down': shift_time,
                'start_times': start_times,
                'cumulative_shift_time': cumulative_shift_time,
                'cumulative_shed_time': cumulative_shed_time,
                'addition': addition,
                'fixes': fixes},
        
        'DLR': {'shift_time': shift_time,
                'ActivateYearLimit': ActivateYearLimit,
                'ActivateDayLimit': ActivateDayLimit,
                'n_yearLimit_shift': n_yearLimit_shift,
                'n_yearLimit_shed': n_yearLimit_shed,
                't_dayLimit': t_dayLimit,
                'addition': addition,
                'fixes': fixes},
        
        'TUD': {'shift_time_down': shift_time,
                'postpone_time': shift_time,
                'annual_frequency_shift': n_yearLimit_shift,
                'annual_frequency_shed': n_yearLimit_shed,
                'daily_frequency_shift': daily_frequency_shift,
                'addition': addition}
                  }
    
    # Optionally attribute half of the potential to a second identical dsm unit with a new label
    if introduce_second_dsm_unit:
        kwargs_all['demand'] = kwargs_all['demand']/2
        kwargs_all['capacity_up'] = kwargs_all['capacity_up']/2
        kwargs_all['capacity_down'] = kwargs_all['capacity_down']/2
        
        # Copy data and drop label as well as inputs since these cause trouble otherwise
        kwargs_all_manipulated = kwargs_all.copy()
        kwargs_all_manipulated.pop('label')
        kwargs_all_manipulated.pop('inputs')
    
    # Actually build the units
    if approach == "DIW":
        demand_dsm = DSM_DIW.SinkDSM(**kwargs_all,
                                     **kwargs_dict[approach])
        
        if introduce_second_dsm_unit:
            demand_dsm2 = DSM_DIW.SinkDSM(label="demand_dsm2",
                                          inputs={b_elec: solph.Flow(variable_costs=0)},
                                          **kwargs_all_manipulated,
                                          **kwargs_dict[approach]
                                    )
        
    elif approach == "IER":
        demand_dsm = DSM_IER.SinkDSI(**kwargs_all,
                                     **kwargs_dict[approach])
        
        if introduce_second_dsm_unit:
            demand_dsm2 = DSM_IER.SinkDSI(label="demand_dsm2",
                                          inputs={b_elec: solph.Flow(variable_costs=0)},
                                          **kwargs_all_manipulated,
                                          **kwargs_dict[approach])
    
    elif approach == "DLR":   
        # Use approach without shifing classes but with shedding
        if not (use_no_shed or use_shifting_classes):
            demand_dsm = DSM_DLR.SinkDR(**kwargs_all,
                                        **kwargs_dict[approach])
            
            if introduce_second_dsm_unit:
                demand_dsm2 = DSM_DLR.SinkDR(label="demand_dsm2",
                                             inputs={b_elec: solph.Flow(variable_costs=0)},
                                             **kwargs_all_manipulated,
                                             **kwargs_dict[approach])
        
        # Use approach without shedding (and without shifting classes)
        elif use_no_shed:
            demand_dsm = DSM_DLR_NoShed.SinkDR(**kwargs_all,
                                               **kwargs_dict[approach])

            if introduce_second_dsm_unit:
                demand_dsm2 = DSM_DLR_NoShed.SinkDR(label="demand_dsm2",
                                                    inputs={b_elec: solph.Flow(variable_costs=0)},
                                                    **kwargs_all_manipulated,
                                                    **kwargs_dict[approach])
        
        # Use approach with shifting classes
        else:
            demand_dsm = DSM_DLR_ShiftClass.SinkDR(**kwargs_all,
                                                    **kwargs_dict[approach])
            
            if introduce_second_dsm_unit:
                demand_dsm2 = DSM_DLR_ShiftClass.SinkDR(label="demand_dsm2",
                                                        inputs={b_elec: solph.Flow(variable_costs=0)},
                                                        **kwargs_all_manipulated,
                                                        **kwargs_dict[approach])

    elif approach == "TUD":
        demand_dsm = DSM_TUD.SinkDSM(**kwargs_all,
                                     **kwargs_dict[approach])

        if introduce_second_dsm_unit:
            demand_dsm2 = DSM_TUD.SinkDSM(label="demand_dsm2",
                                          inputs={b_elec: solph.Flow(variable_costs=0)},
                                          **kwargs_all_manipulated,
                                          **kwargs_dict[approach])
    
    else:
        raise ValueError("No valid value for approach. Must be one of ['DIW', 'IER', 'DLR', 'TUD']")

    # Backup excess / shortage
    excess = solph.Sink(label='excess_el',
                        inputs={b_elec: solph.Flow(variable_costs=1)}
                        )

    s_shortage_el = solph.Source(label='shortage_el',
                                 outputs={
                                     b_elec: solph.Flow(
                                         variable_costs=200)}
                                 )

    # -------------------------- Create Model ----------------------
    
    #pprint.pprint(es.groups, width=1)
    
    # Create Model
    model = solph.Model(es)

    # Solve Model
    model.solve(solver='gurobi', solve_kwargs={'tee': False})

    # Write LP File
    filename = os.path.join(os.path.dirname('__file__'), directory, project +'.lp')
    model.write(filename, io_options={'symbolic_solver_labels': True})

    # Save Results
    es.results['main'] = solph.processing.results(model)
    es.results['meta'] = solph.processing.meta_results(model)
    es.dump(dpath=None, filename=None)

    return model

## Extract model results and plot the model
A function is defined here to extract results from the model and plot the model results.

In [ ]:
def start_model(df_data, timesteps, **kwargs):
    """ Function to extract model results and plot the model. """
    
    approach = kwargs.get('approach', 'DIW')
    
    # Control plotting and processing
    case = kwargs.get('case', 'constant')
    param_variations = kwargs.get('param_variations', '')
    plot = kwargs.get('plot', False)
    figure_size = kwargs.get('figsize', (15, 10))
    legend = kwargs.get("legend", False)
    show = kwargs.get('show', False)
    save = kwargs.get('save', False),
    ax1_ylim = kwargs.get('ax1_ylim', [-10, 210])
    ax2_ylim = kwargs.get('ax2_ylim', [-110, 110])
    include_generators = kwargs.get('include_generators', False)
    
    # Control generation units
    gen_dict = {'include_coal': kwargs.get('include_coal', True),
                'include_gas': kwargs.get('include_gas', False),
                'nom_cap_coal': kwargs.get('nom_cap_coal', 10000),
                'nom_cap_gas': kwargs.get('nom_cap_gas', 10000),
                'nom_cap_pv': kwargs.get('nom_cap_pv', 1)}
    
    # ----------------- Input Data & Timesteps ----------------------------

    # Provide directory
    project = 'demand_shift_' + approach + '_' + case + param_variations
    directory = './'

    # Data manipulation
    data = df_data

    # Timestamp
    datetimeindex = pd.date_range(start='1/1/2013',
                                  periods=timesteps,
                                  freq='H')
    
    len_data = len(data.loc[datetimeindex,:])
    
    # ----------------- DSM Parameterization ----------------------------
    
    introduce_second_dsm_unit = kwargs.get('introduce_second_dsm_unit', False)
    
    # kwargs for all approaches
    delay_time = kwargs.get('delay_time', 1)
    shed_time = kwargs.get('shed_time', 1)
    cost_dsm_up = kwargs.get('cost_dsm_up', 0)
    cost_dsm_down_shift = kwargs.get('cost_dsm_down_shift', 0)
    cost_dsm_down_shed = kwargs.get('cost_dsm_down_shed', 0)
    efficiency = kwargs.get('efficiency', 1)
    shed_eligibility = kwargs.get('shed_eligibility', False)
    shift_eligibility = kwargs.get('shift_eligibility', True)
    
    # kwargs that differ dependent on approach chosen
    shift_time = kwargs.get('shift_time', delay_time/2)
    start_times = kwargs.get('start_times', [1])
    
    # determine recovery / max activations / cumulative shift / shed time dependent on each other
    recovery_time_shift = kwargs.get('recovery_time_shift', None)
    recovery_time_shed = kwargs.get('recovery_time_shed', 24)
    
    if recovery_time_shift is not None:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len_data // (delay_time + recovery_time_shift))
        daily_frequency_shift = kwargs.get('daily_frequency_shift', 
                                           24 // (delay_time + recovery_time_shift))
    else:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len_data // delay_time)
        daily_frequency_shift = kwargs.get('daily_frequency_shift',
                                           24 // delay_time)
   
    if recovery_time_shed is not None:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len_data // (shed_time + recovery_time_shed))
    else:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len_data // shed_time)
    
    cumulative_shift_time = n_yearLimit_shift * shift_time
    cumulative_shed_time = n_yearLimit_shed * shed_time

    if not daily_frequency_shift == 0:
        t_dayLimit = kwargs.get('t_dayLimit', 
                                24 / daily_frequency_shift - 1)
    else:
        t_dayLimit = kwargs.get('t_dayLimit', 0) 
    
    kwargs_dict = {
        'oemof': {'method': kwargs.get('method', None),
                'shift_interval': kwargs.get('shift_interval', None)},

        'DIW': {'method': kwargs.get('method', None),
                'shift_interval': kwargs.get('shift_interval', None)},
                   
        'IER': {'shift_time_up': shift_time,
                'shift_time_down': shift_time,
                'start_times': start_times,
                'cumulative_shift_time': cumulative_shift_time,
                'cumulative_shed_time': cumulative_shed_time,
                'addition': kwargs.get('addition', False),
                'fixes': kwargs.get('fixes', False)},
        
        'DLR': {'shift_time': shift_time,
                'ActivateYearLimit': kwargs.get('ActivateYearLimit', False),
                'ActivateDayLimit': kwargs.get('ActivateDayLimit', False),
                'n_yearLimit_shift': n_yearLimit_shift,
                'n_yearLimit_shed': n_yearLimit_shed,
                't_dayLimit': t_dayLimit,
                'addition': kwargs.get('addition', False),
                'fixes': kwargs.get('fixes', False)},
        
        'TUD': {'shift_time_down': shift_time,
                'postpone_time': shift_time,
                'annual_frequency_shift': n_yearLimit_shift,
                'annual_frequency_shed': n_yearLimit_shed,
                'daily_frequency_shift': daily_frequency_shift,
                'addition': kwargs.get('addition', False)}              
                  }   

    # Introduce another dict for controlling the approaches workflows
    # So far, these only apply for DLR approach (might be removed later on)
    control_dict = {
        'oemof':{},
        'DIW':{},
        'IER':{},
        'DLR':{'use_no_shed': kwargs.get('use_no_shed', True),
               'use_shifting_classes': kwargs.get('use_shifting_classes', False)},
        'TUD':{}      
    }
    
    # ----------------- Create & Solve Model ----------------------------

    # Create model
    model = create_model(data,
                         datetimeindex, 
                         directory, 
                         project,
                         approach,
                         delay_time, 
                         shed_time,
                         cost_dsm_up, 
                         cost_dsm_down_shift,
                         cost_dsm_down_shed,
                         efficiency,
                         shed_eligibility,
                         shift_eligibility,
                         introduce_second_dsm_unit,
                         recovery_time_shift=recovery_time_shift,
                         recovery_time_shed=recovery_time_shed,
                         **kwargs_dict[approach],
                         **control_dict[approach],
                         **gen_dict)
    
    #model.pprint()

    # Get Results
    es = solph.EnergySystem()
    es.restore(dpath=None, filename=None)
    
    results = es.results['main']
    meta = es.results['meta']
    
    # Export data
    df_gesamt = plt_dsm.extract_results(model, approach, **control_dict[approach], **gen_dict)
    #display(df_gesamt)
    
    # write data to csv
    #df_gesamt.to_csv(directory + project + '_data_dump.csv')
    
    # ----------------- Plot Results ----------------------------
    if plot:
        plt_dsm.plot_dsm(
            df_gesamt,
            directory,
            project,
            **control_dict[approach],
            days=2,
            show=show,
            figsize=figure_size,
            ax1_ylim=ax1_ylim,
            ax2_ylim=ax2_ylim,
            include_generators=include_generators,
            approach=approach,
            legend=legend,
            save=save
        )
    
    return df_gesamt, model, meta

## Plot case (availability and generation data)
Function to visualize the case considered for the simple example model
* Show availability, i.e. capacity bounds for DSM
* Show demand before DSM and generation pattern as well as "residual load"

In [ ]:
def plot_case(data, case='constant', **kwargs):
    """ Function to plot the case considered.
    
    Case is defined by availability time series, i.e. capacity bounds for DSM and
    demand before DSM as well as generation pattern.
    """
    show = kwargs.get('show', True)
    save_figs = kwargs.get('save_figs', True)
    
    # Plot demand, wind generation and DR capacity limits
    fig = plt.figure(figsize=(15,4))
    ax = fig.add_subplot(111)

    _ = plt.title('Erzeugung und Last für Fall: "' + case + '"')

    # Define xaxis ticks
    # ax.xaxis_date()
    # ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m - %H h'))  # ('%d.%m-%H h'))
    # ax.set_xlim(data.index.values[0] - pd.Timedelta(1, 'h'), 
    #             data.index.values[0] + pd.Timedelta(1, 'h'))
    # plt.xticks(pd.date_range(start=data.index.values[0], 
    #                          periods=len(data)+1, 
    #                          freq='H'), rotation=90)
    data_reindexed = data.reset_index(drop=True)
    data_reindexed["new_index"] = list(range(1, len(data) + 1))
    data_reindexed.set_index("new_index", drop=True, inplace=True)
    plt.xticks(range(1, len(data) + 1), rotation=90)
    
    ax.plot(data_reindexed.index, data_reindexed['demand_el'].values, drawstyle="steps-post", label="Nachfrage")
    ax.plot(data_reindexed.index, data_reindexed['wind'].values, drawstyle="steps-post", label="Erzeugung")

    # Cap_up and Cap_do only included for proper alignment here
    ax.plot(data_reindexed.index, (data_reindexed['demand_el'] + data_reindexed['Cap_up']).values, 
            drawstyle="steps-post", color="limegreen", label="oberes Limit")
    ax.plot(data_reindexed.index, (data_reindexed['demand_el'] - data_reindexed['Cap_do']).values, 
            drawstyle="steps-post", color="lightcoral", label="unteres Limit")
    
    _ = ax.set_yticks(range(-(data_reindexed.Cap_do.max()-100), data_reindexed.Cap_up.max()+125, 25))
    # ax.legend(bbox_to_anchor=(0., -0.35, 1., 0.102), loc=2, ncol=2, borderaxespad=0.)
    _ = ax.set_xlabel("Zeit in h", labelpad=10)
    _ = ax.set_ylabel("Leistung in MW \n(Nachfrage, Erzeugung, abs. Limits)", labelpad=10)

    plt.grid(alpha=0.6)
    
    # Delta MW on secondary y_axis
    ax2 = ax.twinx()
    #ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m - %H h'))  # ('%d.%m-%H h'))
    #ax2.set_xlim(data.index.values[0] - pd.Timedelta(1, 'h'), 
    #            data.index.values[-1] + pd.Timedelta(1, 'h'))
    #plt.xticks(pd.date_range(start=data.index.values[0], 
    #                         periods=len(data)+1, 
    #                         freq='H'), rotation=90)

    ax2.plot(data_reindexed.index, data_reindexed.Cap_up.values, drawstyle="steps-post", #secondary_y=True, 
             linestyle=":", color="darkgreen", label="verfügbare Kapazität für Lasterhöhung (rechte Achse)")
    ax2.plot(data_reindexed.index, (data_reindexed.Cap_do*-1).values, drawstyle="steps-post", #secondary_y=True, 
             linestyle=":", color="saddlebrown", label="verfügbare Kapazität für Lastreduktion (rechte Achse)")
    
    _ = ax2.set_yticks(range(-data_reindexed.Cap_do.max(), data_reindexed.Cap_up.max()+50, 50))
    handles, labels = [], []
    for ax_object in [ax, ax2]:
        h, l = ax_object.get_legend_handles_labels()
        handles.extend(h)
        labels.extend(l)
        
    _ = plt.legend(
                handles,
                labels,
                loc="upper center",
                bbox_to_anchor=(0, -0.35, 1., 0.102),
                fancybox=True,
                shadow=False,
                ncol=3,
            )
    # ax2.legend(bbox_to_anchor=(0., -0.35, 1., 0.102), loc=1, ncol=1, borderaxespad=0.)
    _ = ax2.set_ylabel("Laständerung in MW \n(Erhöhung, Reduktion)", labelpad=10)
    
    # Do axis aligment
    plt_dsm.align_yaxis(ax, -(data_reindexed.Cap_do.max()-100), ax2, -data.Cap_do.max())
    plt_dsm.align_yaxis(ax, data_reindexed.Cap_up.max()+100, ax2, data.Cap_up.max())
    
    _ = ax.margins(0, 0.05)
    _ = ax2.margins(0, 0.05)
    if show:
        plt.show()

    if save_figs:
        _ = plt.tight_layout()
        name = 'toy-model_' + case + '.png'
        fig.savefig('./graphics/' + name, bbox_inches="tight")
        plt.close()
        print(name + " saved.")

In [ ]:
def plot_case_residual(data, case='constant', **kwargs):
    """ Function to plot the residual load for the respective case. 
    
    Residual load is defined here as the difference between
    generic generation and demand, i.e., what is actually to be balanced.
    """
    show = kwargs.get('show', True)
    save_figs = kwargs.get('save_figs', True)
    
    fig = plt.figure(figsize=(15,4))
    ax = fig.add_subplot(111)
    _ = plt.title('"Residual load" for case "' + case + '"')

    ax.xaxis_date()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m - %H h'))  # ('%d.%m-%H h'))
    ax.set_xlim(data.index.values[0] - pd.Timedelta(1, 'h'), 
                data.index.values[0] + pd.Timedelta(1, 'h'))
    plt.xticks(pd.date_range(start=data.index.values[0], 
                             periods=len(data)+1, 
                             freq='H'), rotation=90)

    ax.plot(data.index, (data['wind'] - data['demand_el']).values, drawstyle="steps-post", 
            linestyle="-.", label="residual load", color="black")
    _ = ax.set_yticks(range(-100,125,25))
    plt.grid()
    _ = ax.set_ylabel("MW \n(residual load)")
    
    if show:
        plt.show()

    if save_figs:
        name = 'toy-model_' + case + '_residual.png'
        fig.savefig('./graphics/' + name)
        plt.close()
        print(name + " saved.")

## Create and display a results table
Function to compare the overall results as far as amount of DSM activations, generation and shortage / excess is concerned.

In [ ]:
def show_results_table(approach_dict, save_results, MultiIndex=False, 
                       param_name="costs", decimals=0):
    """ Show and visualize the results from the previous model run """
    # Show total values for the timeframe considered
    if not MultiIndex:
        dsm = pd.DataFrame()
    else:
        dsm = pd.DataFrame(columns=pd.MultiIndex.from_tuples(approach_dict.keys()))
        dsm.columns.rename(["approach", param_name], inplace=True)
  
    for index, df in approach_dict.items():
        dsm[index]  = df[0].abs().sum().round(decimals=decimals)
        dsm.loc['gen_total', index] = df[0][['wind', 'pv', 'coal1']].sum().sum().round()
        dsm.loc['gen_generic', index] = df[0][['wind', 'pv']].sum().sum().round()

    display(dsm.loc[['demand_el','dsm_tot','excess',
                     'cap_up', 'cap_do',
                     'gen_total','gen_generic',
                     'wind', 'pv', 'coal1']].T.style.bar(axis=0 ,color='goldenrod'))

    if save_results:
        name = 'results_' + case + '.csv'
        dsm.loc[['demand_el','dsm_tot','excess',
                     'cap_up', 'cap_do',
                     'gen_total','gen_generic',
                     'wind', 'pv', 'coal1']].T.to_csv('./graphics/' + name, sep=';', decimal=',')
        print(name + ' saved.')

## Extract model results for realistic toy model
A separate function similar to the one above is defined, which is tailored at a more realistic toy model.
* A loop over all approaches is directly integrated here.
* Results are then added to a dict indexed by approaches.

In [ ]:
def create_realistic_example(data, start_model, timesteps_model, approaches, **kwargs):
    """ Create and solve a more realistic example model. """
    
    dict_df_model = {}
    dict_meta_model = {}
    
    plot = kwargs.get('plot', False)
    
    directory="./"

    # Adjust Timesteps
    start_model = pd.to_datetime(start_model, utc=True).tz_convert(tz)

    if isinstance(timesteps_model, str):
        if timesteps_model == 'all':
            timesteps_model_in = data.index[-1]
        else:
            timesteps_model_in = timesteps_model

        datetimeindex = pd.date_range(start=start_model,
                                      end=timesteps_model_in, freq='H', tz=tz)

    else:
        timesteps_model_in = timesteps_model
        datetimeindex = pd.date_range(start=start_model,
                                      periods=timesteps_model_in, freq='H', tz=tz)
    
    # ----------------- DSM Parameterization ----------------------------
    
    introduce_second_dsm_unit = kwargs.get('introduce_second_dsm_unit', False)
    
    # kwargs for all approaches
    delay_time = kwargs.get('delay_time', 1)
    shed_time = kwargs.get('shed_time', 1)
    cost_dsm_up = kwargs.get('cost_dsm_up', 0)
    cost_dsm_down_shift = kwargs.get('cost_dsm_down_shift', 0)
    cost_dsm_down_shed = kwargs.get('cost_dsm_down_shed', 0)
    efficiency = kwargs.get('efficiency', 1)
    shed_eligibility = kwargs.get('shed_eligibility', False)
    shift_eligibility = kwargs.get('shift_eligibility', True)
    
    # kwargs that differ dependent on approach chosen
    kwargs_dict = {
        'DIW': {'method': kwargs.get('method', None),
                'shift_interval': kwargs.get('shift_interval', None),
                'recovery_time_shift': kwargs.get('recovery_time_shift', None),
                'recovery_time_shed': kwargs.get('recovery_time_shed', 24)},
                   
        'IER': {'shift_time_up': delay_time/2,
                'shift_time_down': delay_time/2,
                'start_times': kwargs.get('start_times', [1]),
                'cumulative_shift_time': len(data.loc[datetimeindex,:]),
                'cumulative_shed_time': len(data.loc[datetimeindex,:]),
                'addition': kwargs.get('addition', False),
                'fixes': kwargs.get('fixes', False)},
        
        'DLR': {'shift_time': delay_time/2,
                'ActivateYearLimit': kwargs.get('ActivateYearLimit', False),
                'ActivateDayLimit': kwargs.get('ActivateDayLimit', False),
                'n_yearLimit_shift': kwargs.get('n_yearLimit_shift', 0),
                'n_yearLimit_shed': kwargs.get('n_yearLimit_shed', 0),
                't_dayLimit': kwargs.get('t_dayLimit', 0),
                'addition': kwargs.get('addition', False),
                'fixes': kwargs.get('fixes', False)},
        
        'TUD': {'shift_time_down': delay_time/2,
                'postpone_time': delay_time/2,
                'annual_frequency_shift': len(data.loc[datetimeindex,:])/delay_time,
                'annual_frequency_shed': len(data.loc[datetimeindex,:])/shed_time,
                'daily_frequency_shift': 24/delay_time,
                'addition': kwargs.get('addition', False)}              
                  }
    
    # Introduce another dict for controlling the approaches workflows
    # So far, these only apply for DLR approach (might be removed later on)
    control_dict = {
        'DIW':{},
        'IER':{},
        'DLR':{'use_no_shed': kwargs.get('use_no_shed', True),
               'use_shifting_classes': kwargs.get('use_shifting_classes', False)},
        'TUD':{}      
    }

    # ----------------- Create & Solve Models ----------------------
    
    for approach in approaches:
        
        model = create_model(data,
                             datetimeindex,
                             directory,
                             project,
                             approach,
                             delay_time, 
                             shed_time, 
                             cost_dsm_up, 
                             cost_dsm_down_shift, 
                             cost_dsm_down_shed, 
                             efficiency,
                             shed_eligibility, 
                             shift_eligibility,
                             introduce_second_dsm_unit,
                             **kwargs_dict[approach],
                             **control_dict[approach])
        
        # Get results
        es = solph.EnergySystem()
        es.restore(dpath=None, filename=None)
        
        results = es.results['main']
        meta = es.results['meta']
        
        # Extract data from model
        df_model = plt_dsm.extract_results(model, approach, **control_dict[approach])
        df_model.index = pd.to_datetime(datetimeindex, utc=True).tz_convert(tz)

        # Export data to dict
        dict_df_model.update({'{}'.format(approach):df_model})
        dict_meta_model.update({'{}'.format(approach):meta})

    return dict_df_model, datetimeindex, dict_meta_model

## Draw results for more realistic toy model
Plot certain columns of the results DataFrame only in order to be able to compare energy on hold, capacity limits etc.

In [ ]:
def draw_results_plot(dict_df_model, start, timesteps, tz, column, negate,
                      figure_size, title, ylabel, drawstyle, color, **kwargs):
    """ Functions draws results plots for realistic toy example """
    
    save = kwargs.get('save', False)
    
    fig, ax = plt.subplots(figsize=figure_size)
    ax.set_title(title, fontsize=15)
    
    keys=[]
    for i, (key, df) in enumerate(dict_df_model.items()):
        keys.append(key)
        if isinstance(column, list):
            for col in column:
                if not negate[col]:
                    df.loc[steps,col].plot(ax=ax, drawstyle=drawstyle, color=color[i])
                else:
                    (df*(-1)).loc[steps,col].plot(ax=ax, drawstyle=drawstyle, color=color[i])
        else:
            if not negate:
                df.loc[steps,column].plot(ax=ax, drawstyle=drawstyle, color=color[i])
            else:
                (df*(-1)).loc[steps,column].plot(ax=ax, drawstyle=drawstyle, color=color[i])

    ax.set_ylabel(ylabel)
    ax.set_xticks(pd.date_range(start=start, periods=timesteps/3, freq='3h', tz=tz))
    ax.xaxis.set_major_formatter((mdates.DateFormatter('%H h')))
    ax.xaxis.grid(True, which="minor")
    ax.xaxis.grid(False, which="major")
    ax.hlines(y=0, xmin=start, xmax=end)

    handles, _ = ax.get_legend_handles_labels() 
    if isinstance(column, list):
        handles = handles[::len(column)]
    labels = ['{}'.format(keys) for keys in keys]

    ax.legend(handles, labels, bbox_to_anchor=(0., -.35, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.1, fontsize=12)
    plt.show()
    
    # Provide directory
    project = 'demand_shift_' + case
    if isinstance(column, list):
        for col in column:
            project = project + '_' + col
    else:
        project = project + '_' + column
    directory = './'
    
    if save:
        fig.set_tight_layout(True)
        name = 'Plot_' + project + '_' + '.png'
        fig.savefig(directory + 'graphics/' + name)
        plt.close()
        print(name + ' saved.')

# Basic comparison for simple setting
In the following, the results for the toy model considerations are depicted. This section is structured as follows:
* At first a base data set is defined, consisting of a flat demand, a fixed generic generation and capacity limits for DSM.
* Then, the demand and/or generic generation the above stated cases are evaluated for the DSM behaviour. The cases are:
    * Case 1: Simple demand variation
    * Case 2: Demand variations
        * 2a: variations with the same amplitude each
        * 2b: variations with a changing amplitude
        * 2c: variations with changed starting timesteps for load shifts
        * 2d: variations with a longer duration (shift for several subsequent hours)
        * 2e: variations demanding for longer shift times
    * Case 3: Increase in demand (use case for load shedding)
    * Case 4: (asymmetric) variations in generation and constant demand
    * Case 4: variations in demand and generation 

## Determine base data set
* A basic data set for the toy model is defined in the following.
* To analyze different behaviour of the modelling approaches, this data set is modified for the different cases.

In [ ]:
timesteps = 48

# base data set
demand = [100] * timesteps
pv = [0] * timesteps
capup = [100] * timesteps
capdo = [100] * timesteps
wind = [100] * timesteps
 
base = [demand, wind, capup, capdo, pv]
df_base = pd.DataFrame(list(zip(*base)))
df_base.rename(columns={0:'demand_el',1:'wind', 2:'Cap_up', 3:'Cap_do', 4:'pv'}, inplace=True)
df_base['timestamp'] = pd.date_range(start='1/1/2013', periods=timesteps, freq='H')
df_base.set_index('timestamp', drop=True, inplace=True)

In [ ]:
df_base.shape

In [ ]:
plot_case(data=df_base, case="Basisfall")
plot_case_residual(data=df_base, case="base data set")

## Case 1: Only one demand variation and constant generation
Analyze one of the most simple examples. Slice only a few timesteps to get an easily readable .lp-file.

In [ ]:
case = 'Simple Nachfragevariation'

# Base data set
df_data = df_base.copy()
demand = [100] * timesteps

# Manipulate demand
demand[1:2] = [150]
demand[5:6] = [50]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Slice the first couple of hours to obtain readable .lp files
if few_timesteps:
    df_data = df_data[:9]

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, 
                                          figsize=(15, 8), legend=True, 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          start_times=[1],
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
approach_dict["DLR"][0]

In [ ]:
show_results_table(approach_dict, save_results)

## Case 2: Variation in demand and constant generation

There are several cases in which the demand pattern is altered in order to analyze the DSM behaviour. Compared to variations in generation (which are basically interchangeable to demand variations if they happen in the opposite direction), the benefit is that deviations in demand can be easily identified in the results plots.

### Case 2a: Variations in demand (same amplitude) and constant generation
Substitute flat demand by introducing two demand peaks and drops each per day with a constant amplitude.

In [ ]:
case = 'Nachfragevariationen mit gleicher Amplitude'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [150]
demand[5:6] = [50]

demand[11:12] = [150]
demand[13:14] = [50]

if timesteps > 24:
    demand[26:27] = [50]
    demand[29:30] = [150]

    demand[36:37] = [50]
    demand[40:41] = [150]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(12, 8),   
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          start_times=[4, 11, 26, 36],
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results)

### Case 2b: Variations in demand (changing amplitude) and constant generation
Substitute flat demand by introducing two demand peaks and drops each per day with a changing, i.e. increasing amplitude.

In [ ]:
case = 'Nachfragevariationen mit unterschiedlicher Amplitude'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[5:6] = [75]

demand[11:12] = [150]
demand[13:14] = [50]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(12,8),  
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          #start_times=[4, 8, 9, 11, 26, 36],
                                          start_times=[4, 11, 26, 36],
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results)

### Case 2c: Variations in demand (changing starts) and constant generation
Substitute flat demand by introducing two demand peaks and drops each per day with a changing amplitude. In addition to that alter the start of the load shifting processes, since some approaches (esp. TUD) tend to behave very sensitive to that.

In [ ]:
case = 'Nachfragevariationen mit veränderten Startzeitpunkten'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[3:4] = [125]
demand[4:5] = [75]

demand[15:16] = [150]
demand[17:18] = [50]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[37:38] = [0]
    demand[41:42] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

> _Note: For this case, an in depth analysis of the starting timesteps used in the IER approach is made._
> _There are five possible constellations for this which can be seen with the second demand variation here serving as an example:_
> 1. _The starting timestep of the demand variation ($t_A=15$) is met: The fluctuation is properly levelled out._
> 2. _Only half of the variation lies within the range from $t_A$ to $t_A + t_{shift}$ ($t_A=12$): The demand response storage level remains unbalanced._
> 3. _A starting timestep within a tolerance band around the actual starting timestep is met (here: $t_A=13$): The fluctuation still can be levelled out since balancing until the end of the demand variation ($t=17$) is possible._
> 4. _There are two starting timesteps ($t_A=12$ and $t_A=15$) surrounding the actual demand variation: An additional activation is made in order to fulfill the balancing for the first and the last starting timestep._
> 5. _Starting timesteps may block the actual activation: This is the case if $t_A=12$, $t_A=13$ and $t_A=16$ have to be met all at the same time._

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(12,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          start_times=[3, 15, 26, 36],
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)
    
    #start_times_dict = {"unbalanced": [3, 12, 26, 36],
    #                    "tolerance": [3, 13, 26, 36],
    #                    "additional": [3, 12, 15, 26, 36],
    #                    "blocking": [3, 12, 13, 16, 26, 36]}
    # Analyze the effect of different start times
    #if approach == "IER":
    #    for st_key, st_val in start_times_dict.items():
    #        approach_dict[approach+"_"+st_key] = \
    #            start_model(df_data, timesteps=len(df_data), 
    #                        plot=True, save=save_figs, case=case, 
    #                        method='delay', delay_time=4, efficiency=1, approach=approach,
    #                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
    #                        cost_dsm_down_shed=cost_dsm_down_shed,
    #                        addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(12,8),  
    #                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
    #                        start_times=st_val,
    #                        fixes=False,
    #                        introduce_second_dsm_unit=introduce_second_dsm_unit,
    #                        use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results)

### Case 2d: Variations in demand (longer duration) and constant generation
Substitute flat demand by introducing in total three demand peaks and drops. The peaks and drops have a longer duration of more than one hour each. Thus, it can be seen how potential limits in interference times (i.e. shift times) behave (for the DLR approach).

> _Note: For the DLR approach, it has been noticed that the core formulation can lead to unbalanced shifts at the end.<br> &rarr; Thus, an own fix is introduced forbidding shifts at the end that cannot be compensated for anymore. This is done within the variant with the addition "\_fixes"._

In [ ]:
case = 'Nachfragevariationen mit längeren Dauern'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[6:8] = [125] * 2
demand[8:10] = [75] * 2

demand[15:18] = [150] * 3
demand[18:21] = [50] * 3

if timesteps > 24:
    demand[34:38] = [25] * 4
    demand[38:42] = [175] * 4

    #demand[34:36] = [50] * 2
    #demand[36:38] = [0] * 2
    #demand[38:40] = [150] * 2
    #demand[40:42] = [200] * 2

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, 
                                          save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, 
                                          figsize=(12,8), 
                                          start_times=[6, 15, 34],
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)
    
    if approach in ["DLR", "IER"]:
        approach_dict[approach+"_fixes"] = start_model(df_data, timesteps=len(df_data), plot=True, 
                                      save=save_figs, case=case, 
                                      method='delay', delay_time=4, efficiency=1, approach=approach,
                                      cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                      cost_dsm_down_shed=cost_dsm_down_shed,
                                      addition=False, fixes=True,
                                      recovery_time_shift=None, recovery_time_shed=4, 
                                      figsize=(12,8), 
                                      start_times=[6, 15, 34],
                                      shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                      introduce_second_dsm_unit=introduce_second_dsm_unit,
                                      use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results)

### Case 2e: Variations in demand (longer shifts) and constant generation
Substitute flat demand by introducing in total three demand peaks and drops. The time between the initial process and its balancing opposite is much longer in this example, so some approaches cannot manage to level out the variations (TUD approach).

In [ ]:
case = 'Nachfragevariationen mit längerem Verschiebehorizont'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[9:10] = [75]

demand[19:20] = [150]
demand[25:26] = [50]

if timesteps > 24:
    demand[34:35] = [25]
    demand[41:42] = [175]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(12,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          start_times=[4, 19, 34],
                                          fixes=True,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results)

## Case 3: Increase in demand and constant generation (use case for load shedding)
Substitute flat demand by introducing one demand peak over several hours which gives a use case for load shedding. Load shedding costs which are usually very high are set to a lower value here, in order to force some shedding avtivity.

In [ ]:
case = 'dem_variation_shed'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[10:14] = [200] * 4

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

> _**NOTE:** A much lower value is used here for shedding costs in order to incentivize load shedding._

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=0.1,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, 
                                          figsize=(15,8), 
                                          fixes=True,
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results)

## Case 4: Variations in generation and constant demand
Introduce wind peaks and wind drops with a changing amplitude. It can be seen that some approaches are not eligible for levelling out changing amplitudes outside fixed shifting cycles (TUD approach).

In [ ]:
case = 'gen_variation'

# Data preperation: manipulate wind data
df_data = df_base.copy()
wind = [100] * timesteps

# wind changes
wind[1:2] = [0]
wind[4:5] = [175]
if timesteps > 24:
    wind[41:42] = [200]
    wind[44:45] = [25]

# Asymmetric shift within shifting cycle possible for TUD
#    wind[1:2] = [0]
#    wind[2:3] = [175]
#    if timesteps > 24:
#        wind[4:5] = [200]
#        wind[5:6] = [25]

df_data['wind'] = wind

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          start_times=[1, 41],
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results)

## Case 5: Variations in demand and generation
Demand and wind variations are introduced. The residual load plot shows what is effectively to be balanced here.

In [ ]:
case = 'combined_variation'

# Data preperation: manipulate demand and wind data
df_data = df_base.copy()
demand = [100] * timesteps
wind = [100] * timesteps

# demand changes
demand[11:12] = [50]
demand[13:14] = [150]
if timesteps > 24:
    demand[33:34] = [150]
    demand[37:38] = [50]

# wind changes
wind[3:4] = [0]
wind[10:11] = [175]
if timesteps > 24:
    wind[38:39] = [200]
    wind[40:41] = [25]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']
df_data['wind'] = wind

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          start_times=[3, 11, 33, 38],
                                          fixes=True,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results)

# Parameters and constraints variations
* Parameters to be variated:
    * delay times
    * shift times
    * costs
    * efficiency
    * longer term energy limits: recovery time and maximum activations
    * short-term energy limits: recovery time and short term limit
* Constraints to be analyzed:
    * optional constraints for Gils (2015) &rarr; included in long- and short-term energy limits analysis
    * overall energy limits (activate / deactive) &rarr; see in long-term energy limits analysis
    * additional constraint (eq. 10 from Zerrahn & Schill 2015)
 
The cases for the parameter and constraints variations are chosen in such a way that the effects of parameter and constraints variations become visible. Therefore, the model setting and constraints formulation has been carefully analyzed in order to find out, how the model setting and individual parameters and constraints should behave.

## Variations in delay time

### Define settings for delay time analysis

In [ ]:
case = 'Variation von Verschiebedauern'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[5:6] = [75]

demand[11:12] = [150]
demand[13:14] = [50]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='Nachfragevariation mit unterschiedlicher Amplitude')
plot_case_residual(data=df_data, case='dem_variation')

### Compare variations in delay time

In [ ]:
# Introduce a dict to store the results of every approach
delay_times = [1, 2, 3, 4, 5]
approach_dict = {}

for dt in delay_times:
    print("-----------------------------------------------------------------------------------------------")
    print("delay time {:d}".format(dt))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:

        approach_dict[(approach, dt)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        param_variations='_delay_time_{:d}'.format(dt),
                        method='delay', delay_time=dt, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(12,8), 
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        start_times=[4, 11, 26, 36],
                        fixes=True,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="delay_time")

## Variations in shift time

### Define settings for shift time analysis

In [ ]:
case = 'dem_variation_duration'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[6:8] = [125] * 2
demand[8:10] = [75] * 2

demand[15:18] = [150] * 3
demand[18:21] = [50] * 3

if timesteps > 24:
    demand[34:38] = [0] * 4
    demand[38:42] = [200] * 4

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='dem_variation_duration')
plot_case_residual(data=df_data, case='dem_variation_duration')

### Compare variations in shift time

In [ ]:
# Introduce a dict to store the results of every approach
shift_times = [1, 2, 3, 4, 5]
approach_dict = {}

for st in shift_times:
    print("-----------------------------------------------------------------------------------------------")
    print("shift time {:d}".format(st))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:

        approach_dict[(approach, st)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        param_variations='_shift_time_{:d}'.format(st),
                        method='delay', delay_time=4, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                        cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        start_times=[6, 15, 34],
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes,
                        shift_time=st)
        
        if approach in ["DLR", "IER"]:
            approach_dict[(approach, st)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            param_variations='_fixes_shift_time_{:d}'.format(st),
                            method='delay', delay_time=4, efficiency=1, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            start_times=[6, 15, 34],
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes,
                            shift_time=st)

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="shift_time")

## Variations in DSM costs

### Define settings for cost analysis

In [ ]:
case = 'dem_variation'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[5:6] = [75]

demand[11:12] = [150]
demand[13:14] = [50]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='dem_variation')
plot_case_residual(data=df_data, case='dem_variation')

### Compare variations in costs

In [ ]:
# Introduce a dict to store the results of every approach
cost_dsm = [0.1, 1, 16.74*2, 16.75*2, 32.5*2, 200]
approach_dict = {}

for cost in cost_dsm:
    
    # Cost is split half on upwards and downwards shift; shedding gets high costs
    cost_dsm_up = cost/2
    cost_dsm_down_shift = cost/2
    cost_dsm_down_shed = 1000 * cost

    print("-----------------------------------------------------------------------------------------------")
    print("costs {:.4f}".format(cost))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:

        approach_dict[(approach, cost)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        param_variations='_dsm_cost_{:f}'.format(cost_dsm_up),
                        method='delay', delay_time=4, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                        start_times=[4, 11, 26, 36],
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="costs")

**Reset costs values:** Reset DSM costs to original values for further considerations

In [ ]:
cost_dsm = 0.1

# Cost is split half on upwards and downwards shift; shedding gets high costs
cost_dsm_up = cost_dsm/2
cost_dsm_down_shift = cost_dsm/2
cost_dsm_down_shed = 1000 * cost_dsm

## Variations in DSM efficiency
So far, only examples with an efficiency of one, i.e. no losses have been considered.<br>
If efficiencies are introduces, load shifts can be balanced out except for these losses.

> _Note:_
> * _For the DLR approach two different variants are considered: the basic one "core" and another one including own fixes. &rarr; Here, the added condition forbidding unbalanced load shifts prevents unexpected behaviour._
> * _The TUD approach does not consider efficiencies and is therefore excluded._

### Define settings for efficiency analysis

In [ ]:
case = 'dem_variation_eff'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[5:6] = [75]

demand[11:12] = [150]
demand[13:14] = [50]
    
if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

    demand[43:44] = [0]
    demand[45:46] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='dem_variation')
plot_case_residual(data=df_data, case='dem_variation')

### Compare variations in efficiency

In [ ]:
# Introduce a dict to store the results of every approach
efficiencies = [1, 0.95, 0.8, 0.2]
approach_dict = {}

for eff in efficiencies:
    
    print("-----------------------------------------------------------------------------------------------")
    print("efficiency {:.2f}".format(eff))
    print("-----------------------------------------------------------------------------------------------")
    
    eff_assigned = eff
    for approach in approaches:
        
        # exclude TUD approach (no efficiencies)
        if not approach == "TUD":
            approach_dict[(approach, eff)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            param_variations='_efficiency_{:f}'.format(eff),
                            method='delay', delay_time=4, efficiency=eff_assigned, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            start_times=[4, 11, 26, 36, 43],
                            addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)
        
        # variant with own fixes
        if approach == "DLR":            
            approach_dict[(approach+"_fixes", eff)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            param_variations='_fixes_efficiency_{:f}'.format(eff),
                            method='delay', delay_time=4, efficiency=eff_assigned, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)          

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="efficiency")

## Variations in recovery time / max. activations

The approaches differ in the way, a recovery time resp. a limit for a given time frame is set. The following elements are used for depicting this.

| approach | parameter used | description |
| ---- | ---- | ---- |
| DIW | recovery time | defines the time between the end of a shifting process and the begin of a subsequent one |
| DLR | $n_{YearLimit}$ | defines the maximum amount of shifts within the optimization timeframe (one year) |
| IER | cumulative shift time | defines the overall time within the optimization timeframe (one year) where shifts in upwards resp. downwards direction may occur |
| TUD | annual frequency shift | defines the maximum amount of shifts within the optimization timeframe (one year) |

The following anologies can be identified which are used to harmonize parameterization:

$$(1) \qquad f_a = n_{YearLimit} = \frac{T} {t_{delay} + t_{recovery}}$$
$$(2) \qquad t_{cum\_shift} = n_{YearLimit} \cdot t_{shift}$$

where

$ f_a $ : annual frequency shift<br>
$ T $ : overall optimization timeframe (in realistic examples usually one year)<br>
$ t_{cum\_shift} $ : cumulative shifting time (pos / neg)

> _Note:_<br>
> _For the DLR approach two different variants are considered: the basic one "core" and another one including own fixes. &rarr; Here, the added conditions preventing shifts which cannot be balanced within the optimization timeframe prevent unexpected behaviour._

### Basic variant
Use the same generation and demand patterns as for most of the other parameter variations above (except for one extra shift at the end).

In [ ]:
# Introduce a dict to store the results of every approach
recovery_time = [None, 2, 4, 16, 44]
approach_dict = {}

for rt in recovery_time:
    
    if rt is not None:
        param_variations='_recovery_time_{:d}'.format(rt)
    else:
        param_variations='_no_recovery_time'

    print("-----------------------------------------------------------------------------------------------")
    if not rt is None:
        print("recovery time {:d}; max. activations: {:.0f}; cumulative shift time {:.0f}".format(rt, 
            len(df_data)//(4+rt), len(df_data)//(4+rt) * 2))
    else:
        print("no limit on recovery time")
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:            
        
        approach_dict[(approach, rt)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        param_variations=param_variations,
                        method='delay', delay_time=4, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, recovery_time_shift=rt, recovery_time_shed=4, figsize=(15,8), 
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        start_times=[4, 11, 26, 36, 43],
                        ActivateYearLimit=True,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True)

### Additional variant
Show dependency of recovery time on consumption pattern for DIW and DLR approach:
* _DIW approach_: Since recovery time constraint is introduced only for upshifts and cap_up, it restricts load shifting further if initial load increases are needed to balance (demand) variations.
* _DLR approach_: Effectively an energy limit is posed for the initial shifts. If all initial shifts are downshifts, the energy limit should become binding.
* This can be identified by adapting case 2b from above in the following way:
    1. The order of the demand variation for the third and fourth use case of load shifting in case 2b is switched, i.e. an initial load decrease is now needed.
    2. Change the amplitudes: Slightly increase it for the first shift cycle and change the amplitudes for second and fourth shifting cycle.

In [ ]:
case = 'dem_variation_switched'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [150]
demand[5:6] = [50]

demand[11:12] = [200]
demand[13:14] = [0]

#demand[17:18] = [200]
#demand[20:21] = [0]

if timesteps > 24:
    demand[26:27] = [150]
    demand[29:30] = [50]

    #demand[31:32] = [150]
    #demand[35:36] = [50]    
    
    demand[36:37] = [175]
    demand[40:41] = [25]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='dem_variation_switched')
plot_case_residual(data=df_data, case='dem_variation_switched')

In [ ]:
# Introduce a dict to store the results of every approach
recovery_time = [None, 2, 4, 16, 44]
approach_dict = {}

for rt in recovery_time:
    
    if rt is not None:
        param_variations='_recovery_time_{:d}'.format(rt)
        param_variations_fixes='_fixes_recovery_time_{:d}'.format(rt)
    else:
        param_variations='_no_recovery_time'
        param_variations_fixes='_fixes_no_recovery_time'

    print("-----------------------------------------------------------------------------------------------")
    if not rt is None:
        print("recovery time {:d}; max. activations: {:.0f}; cumulative shift time {:.0f}".format(rt, 
            len(df_data)//(4+rt), len(df_data)//(4+rt) * 2))
    else:
        print("no limit on recovery time")
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:            
        
        approach_dict[(approach, rt)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        param_variations=param_variations,
                        method='delay', delay_time=4, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                        cost_dsm_down_shed=cost_dsm_down_shed,
                        start_times=[4, 11, 17, 26, 31, 36],
                        addition=False, recovery_time_shift=rt, recovery_time_shed=4, figsize=(15,8), 
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        ActivateYearLimit=True,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)
        
        # variant with own fixes
        if approach == "DLR":            
            approach_dict[(approach+"_fixes", rt)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            param_variations=param_variations_fixes,
                            method='delay', delay_time=4, efficiency=1, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=rt, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            ActivateYearLimit=True,
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True)

### Compare DIW and DLR approach separately (recovery_time and t_dayLimit)
Dependent on how parameterization of the parameters recovery_time from DIW and t_dayLimit from DLR is set, they can be used to model more or less the same thing, i.e. and energy limit imposed over a certain (smaller) timeframe. Hence, an additional comparison is made in the following where only the two approaches are compared to each other.

In [ ]:
# Introduce a dict to store the results of every approach
recovery_time = [2, 4, 16, 44]
approach_dict = {}

for rt in recovery_time:
    
    if rt is not None:
        param_variations='_recovery_time_{:d}'.format(rt)
    else:
        param_variations='_no_recovery_time'

    print("-----------------------------------------------------------------------------------------------")
    if not rt is None:
        print("recovery time resp. t_dayLimit {:d}".format(rt))
    else:
        print("no limit on recovery time resp. t_dayLimit")
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:
        
        if approach in ["DIW", "DLR"]:
        
            approach_dict[(approach, rt)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            param_variations=param_variations,
                            method='delay', delay_time=4, efficiency=1, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=rt, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            ActivateYearLimit=False, ActivateDayLimit=True,
                            t_dayLimit = rt, 
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="recovery_time")

## Variations in amount of daily activations

### Define settings for daily constraint analysis

In [ ]:
case = 'dem_variation_daily'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [175]
demand[5:6] = [25]

demand[11:12] = [200]
demand[13:14] = [0]

demand[17:18] = [200]
demand[19:20] = [0]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

### Compare variations in amount of daily activations

For two approaches, there is a limit on the amount of energy shifted per day.
For the TUD approach from Ladwig (2018) this is a mandatory thing, for the DLR approach from Gils (2015) it is optional.

The following elements are used for depicting this.

| approach | parameter used | description |
| ---- | ---- | ---- |
| DLR | $t_{DayLimit}$ | defines the maximum time for up- resp. downshifts, i.e. the maximum amount of subsequent hours for up- resp. downshifts (not including the current hour) |
| TUD | daily frequency shift | defines the maximum amount of shifts per day |

The following anology can be identified which is used to harmonize parameterization:

$$ \qquad t_{DayLimit} =  \frac{24}{f_d}-1$$

where

$ f_d $ : daily frequency shift<br>

Since the other approaches from DLR and IER do not use a day limit, they are not considered here.

In [ ]:
# Introduce a dict to store the results of every approach
day_limit = [24, 6, 2, 1]
approach_dict = {}

for dl in day_limit:
    
    param_variations='_day_limit_{:d}'.format(dl)

    print("-----------------------------------------------------------------------------------------------")
    print("day limit {:d}; t_DayLimit: {:d}".format(dl, int(24/dl-1)))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:
        if approach in ["DLR", "TUD"]:
        
            approach_dict[(approach, dl)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            param_variations=param_variations,
                            method='delay', delay_time=4, efficiency=1, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            ActivateDayLimit=True, daily_frequency_shift=dl,
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True)

## Activation of optional constraint corresponding to Eq. 10 from Zerrahn and Schill (2015a)
* Set parameter addition to True. Contraint limiting overall amount of DSM capacity (sum in both directions) is introduced.
* delay_time is set to three. Otherwise there would be no simulatneous activation in both directions for the example load and generation pattern (case 2b).

### Define settings for analysis of optional overall capacity limit constraint

In [ ]:
case = 'dem_variation_add'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[5:6] = [125]
demand[6:7] = [75]

demand[13:14] = [150]
demand[15:16] = [50]

if timesteps > 24:
    demand[25:26] = [25]
    demand[28:29] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

### Analyze overall capacity limit constraint

In [ ]:
# Introduce a dict to store the results of every approach
additions = [False, True]
approach_dict = {}

for add in additions:
    print("-----------------------------------------------------------------------------------------------")
    print("constrain overall DSM capacity {0}".format(addition))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:

        approach_dict[(approach, add)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        param_variations='_addition_{0}'.format(addition),
                        method='delay', delay_time=3, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=add, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8),
                        start_times=[5, 13, 25, 36],
                        fixes=True,
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="add_constr")

# Benchmarks
Do some benchmarks for objective value as well as execution times.
The following setting is used:
* A for loop is used in order to create different random data samples.
* A random demand pattern for 168 hours is synthesized.
* A (random) band is created to account for upwards and downwards capacity limits.
* Objective values as well as runtimes are evaluated.

## Objective benchmark

In [ ]:
# suppress pyomo warnings for the iterations
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

### Perform the objective benchmark
Do the actual objective benchmark and store the results into a dict:
* Iterate over a given range (number of iterations)
* Initialize random sample data for demand
* Run the model for each approach and store the results to a dict

In [ ]:
periods = [int(el/2 * 168) for el in range(1, 5, 1)]

In [ ]:
periods

In [ ]:
# for running the benchmark, set boolean variable to True
#target_benchmark=True

In [ ]:
# Do a benchmark for the objective value of the approaches
objective_dict = {}

if target_benchmark:
    
    # generation data is the same for each iteration so it is put outside the loop
    datetimeindex = pd.date_range(start="2013-01-01 00:00", periods=periods[2], freq="H")
    df_bm_data = pd.DataFrame(index=datetimeindex)
    df_bm_data["wind"] = 100
    df_bm_data["pv"] = 0

    for iteration in range(100):
        
        # Create random demand data
        np.random.seed(iteration)
        
        df_bm_data["demand_el"] = 100 + np.random.randint(low=-10, 
                                                          high=10, 
                                                          size=df_bm_data.shape[0]) * 5
        df_bm_data["Cap_up"] = 50 + np.random.randint(low=-5, 
                                                      high=5, 
                                                      size=df_bm_data.shape[0]) * 5
        df_bm_data["Cap_do"] = 50 + np.random.randint(low=-5, 
                                                      high=5, 
                                                      size=df_bm_data.shape[0]) * 5
        
        for approach in approaches:
            #if approach == "IER":
            res = start_model(df_bm_data, timesteps=len(df_bm_data), plot=False, save=False, case=case, 
                              method='delay', delay_time=4, efficiency=1, approach=approach,
                              cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                              cost_dsm_down_shed=cost_dsm_down_shed,
                              addition=False, fixes=True, recovery_time_shift=None, 
                              recovery_time_shed=4, 
                              start_times=list(range(0, len(df_bm_data), 4)),
                              figsize=(15,8), 
                              shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                              introduce_second_dsm_unit=introduce_second_dsm_unit,
                              use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)[2]['objective']
            objective_dict["{}_{}".format(approach, iteration)] = res

### Process objective benchmark results
Evaluate the objective benchmark results:
* Create a dict indexed by approaches only
* Calculate mean values for the objective values from all iterations

In [ ]:
obj_dict = {}
obj_mean_dict = {}

for approach in approaches:
    obj_dict[approach] = {k: v for k, v in objective_dict.items() if approach in k}
    obj_mean_dict[approach] = np.mean(list(obj_dict[approach].values()))

In [ ]:
obj_mean_dict

## Runtime benchmark

### Perform the runtime benchmark
Do the actual runtime benchmark and store the results into a dict:
* Initialize random sample data for demand (same data is used in order to focus on runtime, not influenced by any parameter or solver effects)
* Iterate over a given range (number of iterations) but use the same data in each iteration
* Run the model for each approach and store the solver time results to a dict

In [ ]:
# for running the benchmark, set boolean variable to True
#performance_benchmark=True

In [ ]:
# Do a benchmark for the objective value of the approaches
runtime_dict = {}

if performance_benchmark:
    
    # Data used is the same for each iteration, so it can be set outside the loop
    datetimeindex = pd.date_range(start="2013-01-01 00:00", periods=periods[2], freq="H")
    df_bm_data = pd.DataFrame(index=datetimeindex)
    df_bm_data["wind"] = 100
    df_bm_data["pv"] = 0
    
    np.random.seed(42)

    df_bm_data["demand_el"] = 100 + np.random.randint(low=-10, 
                                                      high=10, 
                                                      size=df_bm_data.shape[0]) * 5
    df_bm_data["Cap_up"] = 50 + np.random.randint(low=-5, 
                                                  high=5, 
                                                  size=df_bm_data.shape[0]) * 5
    df_bm_data["Cap_do"] = 50 + np.random.randint(low=-5, 
                                                  high=5, 
                                                  size=df_bm_data.shape[0]) * 5
    
    # Compare overall execution time
    for approach in approaches:
        #if approach == "IER":
        %timeit -r 10 -n 10 start_model(df_bm_data, timesteps=len(df_bm_data), plot=False, save=False, case=case, method='delay', delay_time=4, efficiency=1, approach=approach, cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed, addition=False, fixes=True, recovery_time_shift=None, recovery_time_shed=4, start_times=list(range(0, len(df_bm_data), 4)), figsize=(15,8), shed_eligibility=True, shed_time=4, n_yearLimit_shed=6, introduce_second_dsm_unit=introduce_second_dsm_unit, use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)
    
    # Compare solver time only (iteration)
    for iteration in range(100):

        for approach in approaches:
            #if approach == "IER":
            res = start_model(df_bm_data, timesteps=len(df_bm_data), plot=False, save=False, case=case, 
                              method='delay', delay_time=4, efficiency=1, approach=approach,
                              cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                              cost_dsm_down_shed=cost_dsm_down_shed,
                              addition=False, fixes=True, recovery_time_shift=None, recovery_time_shed=4,
                              start_times=list(range(0, len(df_bm_data), 4)),
                              figsize=(15,8), 
                              shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                              introduce_second_dsm_unit=introduce_second_dsm_unit,
                              use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes)[2]['solver']['Time']
            runtime_dict["{}_{}".format(approach, iteration)] = res

### Process performance benchmark results
Evaluate the performance benchmark results:
* Create a dict indexed by approaches only
* Calculate mean values for the runtime values from all iterations

In [ ]:
run_dict = {}
run_mean_dict = {}

for approach in approaches:
    run_dict[approach] = {k: v for k, v in runtime_dict.items() if approach in k}
    run_mean_dict[approach] = np.nanmean(list(run_dict[approach].values()))

In [ ]:
run_mean_dict

# More stylized examples
In the following, two other example settings are constructed:
* One very stylized toy model with a given no cost generation and no other generation unit forcing DSM usage
* Another model with a stylized PV infeed and two backup generation units, a coal and a gas power plant. For this example dsm is defined to have costs in between the units, so it should be utilized.

## Very stylized DSM example

In [ ]:
case = 'stylized'

# Base data set
df_data = df_base.copy()
wind = [100] * timesteps

# wind pattern
wind_down = range(200, 25, -25)
wind_up = range(50, 225, 25)

# Manipulate wind
wind[0:len(wind_down)] = list(wind_down)
wind[len(wind_down):len(wind_down)+len(wind_up)] = list(wind_up)

if timesteps > 24:
    wind[24:24+len(wind_up)] = list(wind_up)
    wind[24+len(wind_up):24+len(wind_up)+len(wind_down)] = list(wind_down)
    

df_data['wind'] = wind
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          method='delay', delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, fixes=True,
                                          recovery_time_shift=None, recovery_time_shed=4,
                                          start_times=list(range(0, len(df_data), 4)),
                                          figsize=(15,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes,
                                          include_coal=False)

## PV example

> _Note: This example is similar to the one by Guido Plessmann and Julian Endres given in the oemof.solph documentation, see:_
> * https://oemof.readthedocs.io/en/stable/oemof_solph.html#sinkdsm-custom (accessed 19.08.2020) and
> * https://oemof.files.wordpress.com/2019/12/20191206_dsm_modeling_in_oemof_solph_10th_oemof_dev_meeting_plessmann.pdf (accessed 19.08.2020)

In [ ]:
case="pv_example"

# Create some data
pv_day = [(-(1 / 6 * x ** 2) + 6) / 6 for x in range(-6, 7)]
pv_ts = [0] * 6 + pv_day + [0] * 6
data_dict = {"demand_el": [3] * len(pv_ts),
             "wind": [0] * len(pv_ts),
             "pv": pv_ts,
             "Cap_up": [0.5] * len(pv_ts),
             "Cap_do": [0.5] * len(pv_ts)}
df_data = pd.DataFrame.from_dict(data_dict)
df_data.set_index(df_base.index[:len(df_data)], inplace=True)

In [ ]:
# define cost and capacity parameters (efficiency has to be taken into account)
cost_coal = 13 / 0.4
cost_gas = 25 / 0.4
cost_dsm = (cost_coal+cost_gas)/2
nom_cap_coal = 2.5 / 0.4
nom_cap_gas = 1 / 0.4

Since DSM activation is quite sensitive to costs associated with DSM, this is further analyzed here.

In [ ]:
list(range(0, len(df_data), 4))

In [ ]:
# Introduce a dict to store the results of every approach
cost_dsm = [1, (cost_coal+1)/2, (cost_coal+cost_gas)/2, 1000]
approach_dict = {}

for cost in cost_dsm:
    
    # Cost is split half on upwards and downwards shift; shedding gets high costs
    cost_dsm_up = cost/2
    cost_dsm_down_shift = cost/2
    cost_dsm_down_shed = 1000 * cost

    print("-----------------------------------------------------------------------------------------------")
    print("costs {:.4f}".format(cost))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:
        
        approach_dict[(approach, cost)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        param_variations='_dsm_cost_{:f}'.format(cost_dsm_up),
                        method='delay', delay_time=6, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                        cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, fixes=True,
                        recovery_time_shift=None, recovery_time_shed=4, 
                        start_times=list(range(0, len(df_data), 6)),
                        figsize=(15,8), 
                        shed_eligibility=False, shed_time=4, n_yearLimit_shed=6,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        use_no_shed=use_no_shed, use_shifting_classes=use_shifting_classes,
                        include_coal=True, include_gas=True, 
                        nom_cap_coal=nom_cap_coal, nom_cap_gas=nom_cap_gas, nom_cap_pv=3.5,
                        ax1_ylim = [0, 4], ax2_ylim = [-3, 1], include_generators=True)

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True, decimals=2)

In [ ]:
approach_dict[("IER", 1.0)][0].iloc[6:13].sum()

**Reset costs values:** Reset DSM costs to original values for further considerations

In [ ]:
cost_dsm = 0.1

# Cost is split half on upwards and downwards shift; shedding gets high costs
cost_dsm_up = cost_dsm/2
cost_dsm_down_shift = cost_dsm/2
cost_dsm_down_shed = 1000 * cost_dsm

# More realistic data setting
Input data from the aggregation_example.ipynb written by Julian Endres is used, see: https://github.com/windnode/SinkDSM_example/blob/master/aggregation_example.ipynb

## Initial parameter settings
* Determine model settings (timesteps)
* Only consider project methods (other example covering different aggregation levels is not considered here)

In [ ]:
case = 'realistic'

project = 'methods'
start_model = '2013-03-01 00:00:00+0100'

# Set time range for model (further possible values are uncommented)
#timesteps_model = 'all'
timesteps_model = 24 * 7
#timesteps_model = '2013-03-03 00:00:00'

# Print model start and end information
print('Model begins at:')
print(start_model)

if timesteps_model == 'all':
    print('covers {} timesteps'.format(timesteps_model))
elif isinstance(timesteps_model, int):
    print('ends after {} timesteps'.format(timesteps_model))
elif isinstance(timesteps_model, str):
    print('ends at: {}'.format(timesteps_model))

## Read in input data

In [ ]:
# Input directory and file
input_directory = 'data/{}/'.format(project)
input_file = 'scaled_data_2.csv'
folder = input_directory + input_file
filename_data = os.path.join(os.path.dirname('__file__'), folder)

# Read in data and do timezone adjustments
data = pd.read_csv(filename_data,
                   sep=";",
                   decimal=',',
                   encoding='utf-8',
                   parse_dates=True,
                   date_parser=pd.to_datetime)
data.sort_index(inplace=True)

tz = 'Europe/Berlin'
data.set_index('timestamp', inplace=True)
data.index = pd.to_datetime(data.index, utc=True).tz_convert(tz)
data.rename(columns={'cap_up':'Cap_up', 'cap_do': 'Cap_do'}, inplace=True)

## Extract model results for realistic toy model
Run method defined above

In [ ]:
dict_df_model, datetimeindex, dict_meta_model = \
    create_realistic_example(data, start_model, timesteps_model, approaches,
                             plot=False, save=save_figs, method='delay', 
                             delay_time=4, efficiency=1, 
                             start_times=list(range(0, len(data), 4)),
                             fixes=True,
                             cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                             cost_dsm_down_shed=cost_dsm_down_shed,
                             addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                             shed_eligibility=True, shed_time=4, n_yearLimit_shed=6)

## Define presets and do data preparation for output plots
* Define some settings in order to create another kind of visualization for demand response results.
* Transfer the data in such a shape that it can easily be visualized.

In [ ]:
# Presets
figure_size = (15,6)

# start of plot
start = datetimeindex[0]

# plotted time range
timesteps = 24 * 3 + 3 #*31
steps = pd.date_range(start, periods=timesteps, freq='H', tz='Europe/Berlin') # stepsize

plot_style = 'normal'

color=['teal','goldenrod','grey','coral','olive','orchid', 'silver', 'seagreen', 'slateblue']
drawstyle= {'scientific':{'step':'pre','drawstyle':'steps-post'},'normal':{'step':None,'drawstyle':None }}
ds = drawstyle[plot_style]

tz = 'Europe/Berlin'
end = steps[-1]

# Show some logging info
print('start: {}'.format(start))
print('end:   {}'.format(end))
print('Approaches evaluated:')
print([keys for keys in dict_df_model.keys()])

## Show and plot the results
* Show total values per approach
* Show plots in order to visually inspect differences

In [ ]:
# Show total values for the timeframe considered
dsm = pd.DataFrame()

for index, df in dict_df_model.items():
    dsm[index]  =df.abs().sum().round()
    dsm.loc['gen_total', index] = df[['wind', 'pv', 'coal1']].sum().sum().round()
    dsm.loc['gen_EE', index] = df[['wind', 'pv']].sum().sum().round()

display(dsm.loc[['demand_el','dsm_tot','excess',
                 'cap_up', 'cap_do',
                 'gen_total','gen_EE',
                 'wind', 'pv', 'coal1']].T.style.bar(axis=0 ,color='goldenrod'))

if save_results:
    name = 'results_' + case + '.csv'
    dsm.loc[['demand_el','dsm_tot','excess',
                 'cap_up', 'cap_do',
                 'gen_total','gen_EE',
                 'wind', 'pv', 'coal1']].T.to_csv('./graphics/' + name, sep=';', decimal=',')
    print(name + ' saved.')

**_TODO: Fix plots xaxis_**

In [ ]:
column = ['wind', 'pv', 'coal1']
plots = len([s for s in dict_df_model.keys()])
fig_all, ax = plt.subplots(max(plots,2),1,figsize=(15,plots*4))

i=0
keys=[]
for _, (key, df) in enumerate(dict_df_model.items()):
    
    keys.append(key)
    ax[i].set_title(key)

    # plot demand
    ax[i] = df.loc[steps,'demand_el'].plot(ax=ax[i], drawstyle=ds['drawstyle'], style='--', color='black')
    ax[i] = df.loc[steps,'demand_dsm'].plot(ax=ax[i], drawstyle=ds['drawstyle'], style='--', color='crimson')

    # calc & plot upper capacity
    cap_up = df.loc[steps,'demand_el'] + df.loc[steps,'cap_up']
    cap_up.name='cap_up'
    ax[i] = cap_up.plot(ax=ax[i], drawstyle=ds['drawstyle'], style='-.', color='green')

    # calc & plot lower capacity
    cap_down = df.loc[steps,'demand_el'] - df.loc[steps,'cap_do']
    cap_down.name='cap_do'
    ax[i] = cap_down.plot(ax=ax[i], drawstyle=ds['drawstyle'], style='-.', color='white')
        
    # plot dsm acumulated
    ax[i] = df.loc[steps, 'dsm_acum'].plot(ax=ax[i], drawstyle='steps-post', style='-.', color='indigo')
    
    # area plot for generation units
    upper = 0
    lower = 0
    for gen, col in enumerate(df[column]):
        upper += df.copy().loc[steps, col]
        upper.name = col
        ax[i].fill_between(steps, lower, upper, step=ds['step'], facecolor=color[gen], label=col)
        lower = upper.copy()

    ax[i].set_ylabel('kW')
    ax[i].set_xticks(pd.date_range(start=start, periods=timesteps/3, freq='3h', tz=tz))
    ax[i].xaxis.set_major_formatter((mdates.DateFormatter('%H h')))
    
    #ax[i].xaxis.set_minor_locator(mdates.DayLocator(tz = df.index.tz))
    #ax[i].xaxis.set_minor_formatter(mdates.DateFormatter('%d.%m', tz=df.index.tz))
    #ax[i].xaxis.set_major_formatter(mdates.DateFormatter('%H h', tz=df.index.tz)) 
    #ax[i].xaxis.grid(True, which="minor")
    #ax[i].xaxis.grid(False, which="major")
    #ax[i].set_facecolor='white'
    
    plt.sca(ax[i])
    plt.xticks(rotation=45)
    
    i+=1

handles, labels = ax[i-1].get_legend_handles_labels()
ax[i-1].legend(labels,
             bbox_to_anchor=(0., -.5, 1., .102),
             loc=3, ncol=4,
             mode="expand",
             borderaxespad=0.1,
             facecolor='silver')
fig_all.subplots_adjust(hspace = 0.4)
plt.show()

if save_figs:
    name = 'realistic_example_plot.png'
    fig_all.savefig('./graphics/' + name, bbox_inches='tight')
    print(name + ' saved.')

## Compare energy on hold, capacity limits as well as activations up / down
* Energy on hold: The amount of energy that has to be compensated by load shifts in the other direction again. &rarr; A positive value means that upwards shifts are necessary for balancing.
* Capacity limits: The capacity available for upwards resp. downwards shifts. &rarr; Identical since this is set as an input parameter.
* Activations in upwards / downwards direction.

In [ ]:
draw_results_plot(dict_df_model, start, timesteps, tz, 'dsm_acum', False,
                  figure_size, 'DR - energy on hold', 'kWh', ds['drawstyle'], color, save=save_figs)

In [ ]:
draw_results_plot(dict_df_model, start, timesteps, tz, ['cap_up', 'cap_do'], {'cap_up':False, 'cap_do': True},
                  figure_size, 'DR - capacity limits up / down', 'kW', ds['drawstyle'], color, save=save_figs)

In [ ]:
draw_results_plot(dict_df_model, start, timesteps, tz, ['dsm_up', 'dsm_do_shift'], {'dsm_up':False, 'dsm_do_shift': True},
                  figure_size, 'DR - activations up / down', 'kW', 'steps-post', color, save=save_figs)

# Bibliography

* Gils, Hans Christian (2015): Balancing of Intermittent Renewable Power Generation by Demand Response and Thermal Energy Storage, Stuttgart, http://dx.doi.org/10.18419/opus-6888, accessed 16.08.2019, pp. 67-70.
* Ladwig, Theresa (2018): Demand Side Management in Deutschland zur Systemintegration erneuerbarer Energien, Dresden, https://nbn-resolving.org/urn:nbn:de:bsz:14-qucosa-236074, accessed 02.05.2020, pp. 90-93.
* Steurer, Martin (2017): Analyse von Demand Side Integration im Hinblick auf eine effiziente und umweltfreundliche Energieversorgung, Stuttgart, [10.18419/opus-9181](http://dx.doi.org/10.18419/opus-9181), accessed 17.08.2019, pp. 80-82.
* Zerrahn, Alexander and Schill, Wolf-Peter (2015a): On the representation of demand-side management in power system models, in: Energy (84), pp. 840-845, [10.1016/j.energy.2015.03.037](http://dx.doi.org/10.1016/j.energy.2015.03.037), accessed 16.08.2019, pp. 842-843.
* Zerrahn, Alexander and Schill, Wolf-Peter (2015b): A Greenfield Model to Evaluate Long-Run Power Storage Requirements for High Shares of Renewables, DIW Discussion Papers, No. 1457.